<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Check-mhc-split" data-toc-modified-id="Check-mhc-split-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Check mhc split</a></span><ul class="toc-item"><li><span><a href="#Check-training-set" data-toc-modified-id="Check-training-set-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Check training set</a></span></li><li><span><a href="#Check--validation-set" data-toc-modified-id="Check--validation-set-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Check  validation set</a></span></li></ul></li><li><span><a href="#Check-protein-split" data-toc-modified-id="Check-protein-split-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Check protein split</a></span></li></ul></div>

In [21]:
import numpy as np
import pandas as pd
import pdb
from argparse import ArgumentParser
import shlex
from tqdm import tqdm

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import pMHC
from pMHC import SEP, \
    SPLITS, SPLIT_TRAIN, SPLIT_VAL, SPLIT_TEST, SPLIT_VAL_PROTEINS, SPLIT_TEST_PROTEINS, \
    VIEWS, VIEW_SA, VIEW_SAMA, VIEW_DECONV, \
    INPUT_PEPTIDE, INPUT_CONTEXT
from pMHC.logic import PresentationPredictor
from pMHC.data import from_data, to_input
from pMHC.data.example import Observation
from pMHC.data.mhc_allele import MhcAllele

tqdm.pandas()

In [2]:
pMHC.set_paths(r"C:\Users\tux\Documents\MScProject")

Update project folder to: C:\Users\tux\Documents\project_v2
Load permutation


In [3]:
parser = ArgumentParser()
parser = PresentationPredictor.add_argparse_args(parser)
parser = Trainer.add_argparse_args(parser)


argString = f"--gpus 1 --num_workers 0 "
argString += f"--default_root_dir '{pMHC.OUTPUT_FOLDER}{SEP}' "
argString += f"--track_grad_norm 2 "
argString += f"--proportion 0.1 --precision 16 --datasources 'Edi;Atlas' "
argString += f"--batch_size 32 --learning_rate 0.00001 "
argString += f"--max_seq_length 80 "
argString += f"--input_mode CONTEXT --mhc_rep PSEUDO "
argString += f"--decoys_per_obs 99 --head Attn --max_epochs 1"

args = parser.parse_args(shlex.split(argString))
args_dict = vars(args)
model = PresentationPredictor(**args_dict)


# argString += f"--head Avg "


print(args_dict)

Global seed set to 42


{'name': 'model', 'version': 'version', 'backbone': 'TAPE', 'head': 'Attn', 'head_hidden_features': 512, 'tokenizer': 'Standard', 'input_mode': 'CONTEXT', 'mhc_rep': 'PSEUDO', 'max_seq_length': 80, 'datasources': 'Edi;Atlas', 'decoys_per_obs': 99, 'proportion': 0.1, 'shuffle_data': False, 'seed': 42, 'batch_size': 32, 'learning_rate': 1e-05, 'num_workers': 0, 'output_attentions': False, 'logger': True, 'checkpoint_callback': True, 'default_root_dir': 'C:\\Users\\tux\\Documents\\project_v2\\output\\', 'gradient_clip_val': 0.0, 'gradient_clip_algorithm': 'norm', 'process_position': 0, 'num_nodes': 1, 'num_processes': 1, 'gpus': 1, 'auto_select_gpus': False, 'tpu_cores': None, 'log_gpu_memory': None, 'progress_bar_refresh_rate': None, 'overfit_batches': 0.0, 'track_grad_norm': 2.0, 'check_val_every_n_epoch': 1, 'fast_dev_run': False, 'accumulate_grad_batches': 1, 'max_epochs': 1, 'min_epochs': None, 'max_steps': None, 'min_steps': None, 'max_time': None, 'limit_train_batches': 1.0, 'limit

In [4]:
model.setup()

Global seed set to 42


PresentationPredictor.setup: 2021-07-13 00:07:08
MhcAllele.from_input


MhcAlleles from input: 11074it [00:00, 12787.41it/s]


Protein.from_input


Proteins from input: 185930it [00:14, 13000.74it/s]


...

Decoy.to_input C:\Users\tux\Documents\project_v2\input\decoys\decoys_150000_200000.csv


Decoys: C:\Users\tux\Documents\project_v2\input\decoys\decoys_150000_200000.csv: 4950000it [06:27, 12783.69it/s]


Decoy.to_input C:\Users\tux\Documents\project_v2\input\decoys\decoys_50000_100000.csv


Decoys: C:\Users\tux\Documents\project_v2\input\decoys\decoys_50000_100000.csv: 4950000it [07:31, 10960.03it/s]
Global seed set to 42


PresentationPredictor.setup finished: 2021-07-13 00:37:55


Datasources

OBSERVATIONS                        /   MHC/Obs comb
   Edi                 :  1,550,250 /      7,209,446
   Atlas               :    409,486 /      2,436,897
   TOTAL               :  1,959,736 /      9,646,343




Splits

OBSERVATIONS                          SA            SAMA          Deconv
   train               :           20,659         140,792         673,075
   val                 :            3,230          26,749         127,260
   test                :            5,493          28,432         134,858
   val-prot            :              986           6,943          33,483
   test-prot           :            1,101           6,945          32,751
   val-mhc             :            2,352          20,669          97,844
   test-mhc            :            4,284          20,624          98,040



Dataloaders

EXAMPLES

   train               
      SA               :  2,065,900 /         64,559
      SAM

In [5]:
with open(pMHC.SPLIT_MHC_ALLELES_VAL_FILENAME, 'r') as file:
    val_mhc_allele_groups = file.read()
val_mhc_allele_groups = val_mhc_allele_groups.split("; ")

with open(pMHC.SPLIT_MHC_ALLELES_TEST_FILENAME, 'r') as file:
    test_mhc_allele_groups = file.read()
test_mhc_allele_groups = test_mhc_allele_groups.split("; ")

In [6]:
val_mhc_allele_groups

['HLA-B51',
 'HLA-B53',
 'HLA-B39',
 'HLA-B47',
 'HLA-B08',
 'HLA-A34',
 'HLA-A66',
 'HLA-B42',
 'HLA-B52',
 'HLA-B56']

In [7]:
test_mhc_allele_groups

['HLA-C17',
 'HLA-C01',
 'HLA-C15',
 'HLA-B58',
 'HLA-B37',
 'HLA-A33',
 'HLA-A36',
 'HLA-A74',
 'HLA-B46',
 'HLA-B54']

# Check mhc split

## Check training set

In [8]:
for obs in tqdm(Observation.obs_views[SPLIT_TRAIN][VIEW_SAMA]):
    for mhc_group in [x.name[:7] for x in obs.sample.mhc_alleles]:
        if mhc_group in val_mhc_allele_groups:
            print("Error: in val group")
        if mhc_group in test_mhc_allele_groups:
            print("Error: in test group")

100%|██████████████████████████████████████████████████████████████████████| 140792/140792 [00:00<00:00, 197490.70it/s]


## Check  validation set

In [9]:
for obs in tqdm(Observation.obs_views[SPLIT_VAL][VIEW_SAMA]):
    for mhc_group in [x.name[:7] for x in obs.sample.mhc_alleles]:
        if mhc_group in test_mhc_allele_groups:
            print("Error: in test group")

100%|████████████████████████████████████████████████████████████████████████| 26749/26749 [00:00<00:00, 270795.87it/s]


# Check protein split

In [24]:
proteins_train = []
peptides_train = []
for obs in tqdm(Observation.obs_views[SPLIT_TRAIN][VIEW_SAMA]):
    proteins_train += [x.name for x in obs.peptide.proteins]
    peptides_train.append(obs.peptide.seq)
    
proteins_val = []
peptides_val = []
for obs in tqdm(Observation.obs_views[SPLIT_VAL_PROTEINS][VIEW_SAMA]):
    proteins_val += [x.name for x in obs.peptide.proteins]
    peptides_val.append(obs.peptide.seq)
    
proteins_test = []
peptides_test = []
for obs in tqdm(Observation.obs_views[SPLIT_TEST_PROTEINS][VIEW_SAMA]):
    proteins_test += [x.name for x in obs.peptide.proteins]
    peptides_test.append(obs.peptide.seq)
    
proteins_train = set(proteins_train)
peptides_train = set(peptides_train)
proteins_val = set(proteins_val)
peptides_val = set(peptides_val)
proteins_test = set(proteins_test)
peptides_test = set(peptides_test)

100%|██████████████████████████████████████████████████████████████████████████| 6945/6945 [00:00<00:00, 435248.50it/s]


In [26]:
print(len(proteins_val.intersection(proteins_train)))
print(len(proteins_test.intersection(proteins_train)))
print(len(proteins_test.intersection(proteins_val)))

0
0
0


In [27]:
print(len(peptides_val.intersection(peptides_train)))
print(len(peptides_test.intersection(peptides_train)))
print(len(peptides_test.intersection(peptides_val)))

0
0
0
